In [ ]:
%matplotlib qt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib
matplotlib.rcParams.update({'font.size': 25})

In [ ]:
#READ DATA FROM FILE AND EXEND THE VALUES LIST
def getValues(folder, mode, pose, label, values, labels, readIMU):
    valuesRMS = []
    with open('../_readings/' + folder + '/' + mode + '/RMS'+ pose +'.txt') as f:
        linesRMS = f.readlines()
    linesRMS = list(map(float, ([x.strip() for x in linesRMS])))
    for i in range(0, len(linesRMS), 8):
        valuesRMS.append(np.asarray(linesRMS[i:i+8]) / 128)
        labels.append(label)
    valuesAll = np.asarray(valuesRMS)

    if readIMU:
        valuesIMU = []
        with open('../_readings/' + folder + '/' + mode + '/IMU'+ pose +'.txt') as f:
            linesIMU = f.readlines()
        linesIMU = list(map(float, ([x.strip() for x in linesIMU])))
        for i in range(0, len(linesIMU), 3):
            valuesIMU.append(np.asarray(linesIMU[i:i+3]) / (np.pi * 2) + 0.5)
        valuesIMU = np.asarray(valuesIMU)
        valuesRoll = valuesIMU[:, 0][:, None]
        valuesAll = np.append(valuesAll, valuesRoll, axis=1)

    values = np.append(values, valuesAll, axis=0)
    return [values, labels]

#PREPARE VALUES
def prepareValues(folders, mode, poses, labelsIn, valuesOut, labelsOut, readIMU):
    for folder in folders:
        for i in labelsIn:
            [valuesOut, labelsOut] = getValues(folder, mode, poses[i], i, valuesOut, labelsOut, readIMU)
    valuesOut = np.asarray(valuesOut)
    labelsOut = np.asarray(labelsOut)
    print(valuesOut.shape)
    return [valuesOut, labelsOut]

In [ ]:
#GET VALUES
readIMU = True

#train data
trainFolders = ['2018_8_24_aljaz', '2018_8_25_aljaz', '2018_8_26_aljaz', 'branko', 'martin', 'martina', 'saso', 'tadej']
trainPoses = ['extension', 'flexion', 'radDev', 'ulDev', 'pron', 'sup', 'palmOut', 'fist', 
              'hibernation', 'hibernation', 'hibernation']
trainLabelsIn = [0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8]
[trainValues, trainLabels] = prepareValues(trainFolders, '1', trainPoses, trainLabelsIn, 
                                           np.empty((0, 9 if readIMU else 8)), [], readIMU)

#validation data
validationFolders = ['2018_8_24_aljaz', '2018_8_25_aljaz', '2018_8_26_aljaz', 'branko', 'martin', 'martina', 'saso', 'tadej']
validationPoses = ['extension', 'flexion', 'radDev', 'ulDev', 'pron', 'sup', 'palmOut', 'fist', 
                   'hibernation', 'hibernation', 'hibernation']
validationLabelsIn = [0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8]
[validationValues, validationLabels] = prepareValues(validationFolders, '2', validationPoses, validationLabelsIn,
                                                     np.empty((0, 9 if readIMU else 8)), [], readIMU)

#test data
testFolders = ['2018_8_24_aljaz', '2018_8_25_aljaz', '2018_8_26_aljaz', 'branko', 'martin', 'martina', 'saso', 'tadej']
testPoses = ['extension', 'flexion', 'radDev', 'ulDev', 'pron', 'sup', 'palmOut', 'fist', 
             'hibernation']
testLabelsIn = [0, 1, 2, 3, 4, 5, 6, 7, 8]
[testValues, testLabels] = prepareValues(testFolders, '3', testPoses, testLabelsIn, 
                                         np.empty((0, 9 if readIMU else 8)), [], readIMU)

In [ ]:
#TRAINING
maxEpochs = 9001
earlyQuit = True
quitPerc = 0.25

#model + compilation
model = keras.Sequential([
    keras.layers.Dense(200, activation = tf.nn.sigmoid, input_shape = (trainValues.shape[1],)),
    keras.layers.Dense(100, activation = tf.nn.sigmoid),
    keras.layers.Dense(50, activation = tf.nn.sigmoid),
    keras.layers.Dense(9, activation = tf.nn.sigmoid)
])
model.compile(optimizer = tf.keras.optimizers.Adam(), 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

#actual training
start = time.time()

accs = []
valAccs = []
losses = []
valLosses = []

bestModel = None
bestValLoss = 0

for i in range(0, maxEpochs):
    print('EPOCH:', i + 1, '/', maxEpochs)

    history = model.fit(x = trainValues, 
                        y = trainLabels, 
                        validation_data = (validationValues, validationLabels))
    
    accs.append(history.history.get('acc')[0])
    valAccs.append(history.history.get('val_acc')[0])
    losses.append(history.history.get('loss')[0])
    valLosses.append(history.history.get('val_loss')[0])
    
    # check if model is better
    if bestModel == None:
        bestModel = model
        bestValLoss = valLosses[-1]
    elif bestValLoss > valLosses[-1]:
        bestModel = model
        bestValLoss = valLosses[-1]
        
    #quit if critera is met
    if earlyQuit and valLosses[-1] / bestValLoss > 1 + quitPerc:
        break
    
model = bestModel
    
end = time.time()

#calculate time spent training
totalSecs = end - start
totalMins = int(totalSecs / 60)
hours = int(totalMins / 60)
mins = int(totalMins % 60)
secs = int(totalSecs % 60)

print("Training time: " + str(hours) + " h " + str(mins) + " min " + str(secs) + " s")

In [ ]:
#PLOT TEST
predictions = model.predict(testValues)
plt.close('all')
plt.ylabel('Odziv nevronske mreže')
plt.xlabel('Zaporedna številka vzorca testne množice')
plt.plot(predictions)
leg = plt.gca().legend(['Ekstenzija', 'Fleksija', 'Radialna deviacija', 'Ulnarna deviacija', 
                        'Pronacija', 'Supinacija', 'Iztegnjena dlan', 'Pest', 'Mirovanje'], 
                        loc=2, bbox_to_anchor = (1, 1), frameon=False)
for legobj in leg.legendHandles:
    legobj.set_linewidth(10)
plt.gca().set_position([plt.gca().get_position().x0, 
              plt.gca().get_position().y0, 
              plt.gca().get_position().width * 0.8, 
              plt.gca().get_position().height])

In [ ]:
#PLOT ACCURACIES AND LOSSES
plt.close('all')

plt.figure()
plt.ylabel('Natančnost nevronske mreže')
plt.xlabel('Zaporedna številka epohe')
plt.plot(accs)
plt.plot(valAccs)
leg = plt.gca().legend(['Učna\nmnožica', 'Validacijska\nmnožica'], 
                        loc=2, bbox_to_anchor = (1, 1), frameon=False)
plt.gca().set_position([plt.gca().get_position().x0, 
              plt.gca().get_position().y0, 
              plt.gca().get_position().width * 0.8, 
              plt.gca().get_position().height])
for legobj in leg.legendHandles:
    legobj.set_linewidth(10)

plt.figure()
plt.ylabel('Izguba nevronske mreže')
plt.xlabel('Zaporedna številka epohe')
plt.plot(losses)
plt.plot(valLosses)
leg = plt.gca().legend(['Učna\nmnožica', 'Validacijska\nmnožica'], 
                        loc=2, bbox_to_anchor = (1, 1), frameon=False)
plt.gca().set_position([plt.gca().get_position().x0, 
              plt.gca().get_position().y0, 
              plt.gca().get_position().width * 0.8, 
              plt.gca().get_position().height])
for legobj in leg.legendHandles:
    legobj.set_linewidth(10)

In [ ]:
#PLOT CLASSIFICATION
#thresh = 1e-6
classifications = []
for i in range(0, predictions.shape[0]):
    row = predictions[i, :]
    #if row[-1] > thresh:
    #    maxInd = row.shape[0] - 1
    #else:
    maxInd = np.argmax(row)
    newRow = np.zeros(predictions.shape[1])
    newRow[maxInd] = 1
    classifications.append(newRow)
plt.figure()
plt.plot(classifications)

In [ ]:
#EVALUATE ON TEST DATA
testLoss, testAcc = model.evaluate(testValues, testLabels)
print(testAcc)

In [ ]:
#LOAD MODEL
model = keras.models.load_model('../_models/model')
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#SAVE MODEL
model.save('../_models/model', include_optimizer=False)